# Building an Image Classifier and Integrating it into Web Applications Using AI Squared

In this demo, we are going to create an image classification model using TensorFlow and BeyondML and integrate it into an `.air` file to be consumed using the AI Squared Platform and Browser Extension

To do this, we are first going to install the `aisquared` and `beyondml` packages from `PyPI`, then we are going to use the `CIFAR10` dataset from TensorFlow to build an image classification model. Finally, we are going to compile the model and all required configuration information into a `.air` file to be uploaded into the AI Squared platform.

# Installing and Importing the Required Packages

Before we continue, we have to `aisquared` and `beyondml` packages. Then, we need to import the packages we are going to need to build and train the model and create the `.air` file.

In [ ]:
# Install the required packages
! pip install aisquared
! pip install beyondml

In [ ]:
# Import required packages

import tensorflow as tf
from beyondml import tflow
import aisquared
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Loading and Preprocessing the Data and Building the Model

Now that we have all of the required packages installed and imported, it's time to get to work!  The following code cells do the following:

- Loads the CIFAR10 dataset - an image classification dataset using a subset of the ImageNet dataset with ten categories
- Preprocesses the dataset by dividing all pixel values by 255
- Builds our simple convolutional model
- Prunes and optimizes the model using BeyondML
- Trains the model
- Presents model results on test (hold-out) data

### Load and preprocess the data

In [ ]:
# Load the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()


# Preprocess the data
x_train = x_train/255
x_test = x_test/255

### Build and prune the model

In [ ]:
# Build the model
input_layer = tf.keras.layers.Input(x_train.shape[1:])
x = tflow.layers.MaskedConv2D(16, activation = 'relu', padding = 'same')(input_layer)
x = tf.keras.layers.BatchNormalization()(x)
x = tflow.layers.MaskedConv2D(16, activation = 'relu', padding = 'same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tflow.layers.MaskedConv2D(32, activation = 'relu', padding = 'same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tflow.layers.MaskedConv2D(32, activation = 'relu', padding = 'same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tflow.layers.MaskedConv2D(64, activation = 'relu', padding = 'same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tflow.layers.MaskedConv2D(64, activation = 'relu', padding = 'same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tflow.layers.MaskedDense(128, activation = 'relu')(x)
output_layer = tflow.layers.MaskedDense(10, activation = 'softmax')(x)

model = tf.keras.models.Model(input_layer, output_layer)

# Prune the model using BeyondML so it is more performant
model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')
model = tflow.utils.mask_model(
    model,
    percentile = 75,
    x = x_train[:1000],
    y = y_train[:1000].reshape(-1, 1)
)
model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

# Print out the model summary
model.summary()

### Train the model

In [ ]:
# Fit the model on training data
model.fit(
    x_train,
    y_train,
    validation_split = 0.2,
    batch_size = 256,
    epochs = 10,
    callbacks = tf.keras.callbacks.EarlyStopping(min_delta = 0.004, patience = 2, restore_best_weights = True)
)

# Remove the layer masks so that the model is fully optimized
model = tflow.utils.remove_layer_masks(model)

### Present model performance

In [ ]:
preds = model.predict(x_test).argmax(axis = 1)
print('Performance on Test Data:')
print('\n\n')
print('Confusion Matrix')
print('\n')
print(confusion_matrix(y_test, preds))
print('\n\nClassification Report\n')
print(classification_report(y_test, preds))
model.save('cifar10_classifier.h5')

# Creating a `.air` File to use the Model

Now that we have created the model and are happy with its performance, it's time to create a `.air` file that will use the model in the browser! To do this, we have to use the `aisquared` package, specifically the `aisquared.config` subpackage, to build out a `ModelConfiguration` and compile the `.air` file.

The following cells show the code to do just that.

In [ ]:
# Label map that maps class numbers to actual predicted classes
label_map = [
                'airplane',
                'automobile',
                'bird',
                'cat',
                'deer',
                'dog',
                'frog',
                'horse',
                'ship',
                'truck'
            ]

# Harvester that tells the AI Squared platform to harvest all images in the webpage
harvester = aisquared.config.harvesting.ImageHarvester()

# Preprocesser that reshapes the images to the required image size and also divides all pixel values by 255
preprocesser = aisquared.config.preprocessing.image.ImagePreprocessor(
    [
        aisquared.config.preprocessing.image.Resize([x_train.shape[1], x_test.shape[2]]),
        aisquared.config.preprocessing.image.DivideValue(255)
    ]
)

# Analytic that tells the AI Squared platform to run the model that we have just saved
analytic = aisquared.config.analytic.LocalModel('cifar10_classifier.h5', 'cv')

# Postprocess results to map to the class names for the dataset
postprocesser = aisquared.config.postprocessing.MulticlassClassification(label_map)

# Render results back on the images
renderer = aisquared.config.rendering.ImageRendering(font_size = '10', thickness = '10', font_color = 'white')

# Provide feedback form
feedback = aisquared.config.feedback.MulticlassFeedback(label_map)

# Create the ModelConfiguration object to hold all of the configuration information
model_config = aisquared.config.ModelConfiguration(
    'CIFAR10 Classifier',
    harvester,
    preprocesser,
    analytic,
    postprocesser,
    renderer,
    feedback
)

# Compile the configuration into the .air file, including quantizing the weights of the model to 16-bit floats
model_config.compile(dtype = 'float16')

# Done!

Now you should have a file named `CIFAR10 Classifier.air` in this directory!  Congrats on creating the `.air` file - now you can go and upload that file to the AI Squared platform and see it working!